질문

"라면 끓일 때 매번 라면 스프부터 직접 만든다면 얼마나 힘들까요?"

모듈은 이미 만들어진 ‘기능 스프’ 같은 것

다른 사람이 만들어 둔 기능(코드)을 그대로 가져와서 [재사용] 가능

덕분에 시간 절약, 버그 줄이기, 협업 용이

# 모듈의 종류
- 표준 모듈: 파이썬이 기본 제공 (math, datetime, os 등)

- 외부 모듈: pip install로 설치 (requests, pandas, flask 등)

- 사용자 정의 모듈: 내가 만든 .py 파일을 불러오는 것

```python
import 모듈명
import 모듈명 as 별명
from 모듈명 import 함수명
from 모듈명 import 함수명 as 별명

In [1]:
from mymath import add

# 모듈.함수명

In [3]:
# math 모듈 사용
import math

print("제곱근:", math.sqrt(16))
print("원주율:", math.pi)

# datetime 모듈 사용
from datetime import date
print("오늘 날짜:", date.today())

제곱근: 4.0
원주율: 3.141592653589793
오늘 날짜: 2025-08-09


# 포인트

- 표준 모듈은 설치 없이 바로 사용 가능

- 필요한 함수만 불러올 수도 있음 (from ~ import ~)

# 외부 모듈
- pip install requests

```python
import requests

# API 대신 httpbin.org로 테스트
response = requests.get("https://httpbin.org/get")
print("상태 코드:", response.status_code)
print("응답 JSON:", response.json())

# 포인트

- pip install로 외부 패키지를 설치

- 설치 후 import로 불러와 사용

- 네트워크, 데이터 분석, 웹 개발 등 파이썬이 기본 제공하지 않는 기능도 쉽게 확장 가능

In [9]:
import socket
import json

def recv_all(sock):
    chunks = []
    while True:
        b = sock.recv(4096)
        if not b:
            break
        chunks.append(b)
    return b"".join(chunks)

def parse_headers(header_bytes):
    header_text = header_bytes.decode("iso-8859-1")  # 헤더는 라틴-1로 안전
    lines = header_text.split("\r\n")
    status_line = lines[0]
    headers = {}
    for line in lines[1:]:
        if not line:
            continue
        k, v = line.split(":", 1)
        headers[k.strip().lower()] = v.strip()
    # HTTP/1.1 200 OK
    proto, status, *_ = status_line.split(" ", 2)
    return int(status), headers

def read_chunked(body_bytes):
    i = 0
    out = bytearray()
    while True:
        # chunk size line
        j = body_bytes.find(b"\r\n", i)
        if j == -1:
            raise ValueError("Invalid chunked encoding: no CRLF after size")
        size_line = body_bytes[i:j].split(b";", 1)[0]  # ignore extensions
        size = int(size_line, 16)
        i = j + 2
        if size == 0:
            # optional trailer headers until CRLF
            k = body_bytes.find(b"\r\n\r\n", i)
            if k == -1:
                # no trailers
                return bytes(out)
            return bytes(out)
        # read chunk data + CRLF
        out += body_bytes[i:i+size]
        i += size
        if body_bytes[i:i+2] != b"\r\n":
            raise ValueError("Invalid chunked encoding: missing CRLF after chunk")
        i += 2

def get(host, path, port=80, https=False):
    # 이 예시는 http만 다룹니다(간소화). https는 ssl.wrap_socket 필요.
    addr = (host, port)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect(addr)
        req = (
            f"GET {path} HTTP/1.1\r\n"
            f"Host: {host}\r\n"
            "User-Agent: tiny-socket-client/0.1\r\n"
            "Accept: application/json\r\n"
            "Connection: close\r\n"
            "\r\n"
        )
        sock.sendall(req.encode("ascii"))
        raw = recv_all(sock)

    # 헤더/바디 분리
    sep = raw.find(b"\r\n\r\n")
    if sep == -1:
        raise ValueError("Invalid HTTP response (no header/body separator)")
    header_bytes = raw[:sep]
    body_bytes = raw[sep+4:]

    status, headers = parse_headers(header_bytes)

    if status != 200:
        raise RuntimeError(f"HTTP {status}")

    # 바디 조립 (chunked vs content-length vs EOF)
    te = headers.get("transfer-encoding")
    if te and "chunked" in te.lower():
        body = read_chunked(body_bytes)
    else:
        cl = headers.get("content-length")
        if cl is not None:
            need = int(cl)
            # Connection: close로 대부분 raw에 다 들어와 있지만,
            # 일반화하려면 부족분을 재수신해야 함. 여기선 간소화 가정.
            body = body_bytes[:need]
        else:
            # 길이 정보 없으면 EOF까지가 바디(이미 recv_all로 끝까지 받음)
            body = body_bytes

    # 문자셋 추정
    ctype = headers.get("content-type", "")
    charset = "utf-8"
    if "charset=" in ctype:
        charset = ctype.split("charset=")[-1].split(";")[0].strip()

    text = body.decode(charset, errors="replace")

    # JSON 파싱 (실패 시 예외)
    return json.loads(text)

if __name__ == "__main__":
    data = get("httpbin.org", "/json")
    print(data)


{'slideshow': {'author': 'Yours Truly', 'date': 'date of publication', 'slides': [{'title': 'Wake up to WonderWidgets!', 'type': 'all'}, {'items': ['Why <em>WonderWidgets</em> are great', 'Who <em>buys</em> WonderWidgets'], 'title': 'Overview', 'type': 'all'}], 'title': 'Sample Slide Show'}}


In [ ]:
!pip install requests

import requests

data = requests.get("https://httpbin.org/json").json()
print(data)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
{'slideshow': {'author': 'Yours Truly', 'date': 'date of publication', 'slides': [{'title': 'Wake up to WonderWidgets!', 'type': 'all'}, {'items': ['Why <em>WonderWidgets</em> are great', 'Who <em>buys</em> WonderWidgets'], 'title': 'Overview', 'type': 'all'}], 'title': 'Sample Slide Show'}}



```python
# mymath.py
def add(a, b):
    return a + b

def sub(a, b):
    return a - b

```python
import mymath

print(mymath.add(3, 5))
print(mymath.sub(10, 4))

In [5]:
import mymath

print(mymath.add(3, 5))
print(mymath.sub(10, 4))

8
6


# 섭씨를 화씨로 바꾸는 myconverter.py를 만들어서 
- 30도 섭씨를 화씨로 바꾸고
- 80도 화씨를 섭씨로 바꿀것

```python
def c_to_f(c):
    return c * 9/5 + 32

def f_to_c(f):
    return (f - 32) * 5/9